In [1]:
!pip install optuna

In [2]:
# for hyper parameter

In [3]:
!pip install xgboost

In [4]:
import optuna
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [6]:
df = pd.read_csv('https://raw.githubusercontent.com/Jhoie/Admission-Prediction/main/Admission_Predict.csv')

In [7]:
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
395,396,324,110,3,3.5,3.5,9.04,1,0.82
396,397,325,107,3,3.0,3.5,9.11,1,0.84
397,398,330,116,4,5.0,4.5,9.45,1,0.91
398,399,312,103,3,3.5,4.0,8.78,0,0.67


In [8]:
df.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,200.500000,316.807500,107.410000,3.087500,3.400000,3.452500,8.598925,0.547500,0.724350
std,115.614301,11.473646,6.069514,1.143728,1.006869,0.898478,0.596317,0.498362,0.142609
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.000000,6.800000,0.000000,0.340000
25%,100.750000,308.000000,103.000000,2.000000,2.500000,3.000000,8.170000,0.000000,0.640000
50%,200.500000,317.000000,107.000000,3.000000,3.500000,3.500000,8.610000,1.000000,0.730000
75%,300.250000,325.000000,112.000000,4.000000,4.000000,4.000000,9.062500,1.000000,0.830000
max,400.000000,340.000000,120.000000,5.000000,5.000000,5.000000,9.920000,1.000000,0.970000


In [9]:
df.isna().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [10]:
df['GRE Score'] = df['GRE Score'].fillna(df['GRE Score'].mean())
df['TOEFL Score'] = df['TOEFL Score'].fillna(df['TOEFL Score'].mean())
df['University Rating'] = df['University Rating'].fillna(df['University Rating'].mean())

In [12]:
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
395,396,324,110,3,3.5,3.5,9.04,1,0.82
396,397,325,107,3,3.0,3.5,9.11,1,0.84
397,398,330,116,4,5.0,4.5,9.45,1,0.91
398,399,312,103,3,3.5,4.0,8.78,0,0.67


In [19]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1]


In [22]:
x = x.drop(columns=['Serial No.'])

In [23]:
y

0      0.92
1      0.76
2      0.72
3      0.80
4      0.65
       ... 
395    0.82
396    0.84
397    0.91
398    0.67
399    0.95
Name: Chance of Admit , Length: 400, dtype: float64

In [24]:
from sklearn.preprocessing import StandardScaler
std_sca = StandardScaler()
x   = std_sca.fit_transform(x)

In [26]:
from sklearn.metrics import mean_squared_error
def objective(trail , data = x , target = y ):
    train_x , test_x ,train_y , test_y = train_test_split(data , target , test_size = .10,random_state = 30  )
    param = {
        'tree_method':'gpu_hist',
        'lambda':trail.suggest_loguniform('lambda' , 1e-4,10.0),
        'alpha' :trail.suggest_loguniform('alpha' , 1e-4 , 10.0),
        'colsample_bytree' :trail.suggest_categorical('colsample_bytree' , [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
        'subsample' :trail.suggest_categorical('subsample' , [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
        'learning_rate' : trail.suggest_categorical('learning_rate' , [.00001,.0003,.008,.02,.01,1,8]),
        'n_estimator' :30000,
        'max_depth' :trail.suggest_categorical('max_depth', [3,4,5,6,7,8,9,10,11,12]),
        'random_state' :trail.suggest_categorical('random_state' , [10,20,30 ,2000 , 3454,243123]),
        'min_child_weight' :trail.suggest_int('min_child_weight' ,1,200)
    }
    xgb_reg_model = xgb.XGBRegressor(**param)
    xgb_reg_model.fit(train_x,train_y,eval_set = [(test_x,test_y)] , verbose = True )
    pred_xgb = xgb_reg_model.predict(test_x)
    rmse = mean_squared_error(test_y,pred_xgb)
    return rmse

{'lambda': 0.5646022891371127,
 'alpha': 0.35848071012137483,
 'colsample_bytree': 0.9,
 'subsample': 0.2,
 'learning_rate': 1,
 'max_depth': 11,
 'random_state': 30,
 'min_child_weight': 30}

In [27]:
!NVIDIA-SMI

'NVIDIA-SMI' is not recognized as an internal or external command,
operable program or batch file.


In [28]:
find_param = optuna.create_study(direction='minimize')
find_param.optimize(objective,n_trials = 10)
find_param.best_trial.params

[I 2023-05-20 22:19:43,489] A new study created in memory with name: no-name-3a904f89-2e12-4e1f-8715-847a678102e6
[W 2023-05-20 22:19:43,530] Trial 0 failed because of the following error: XGBoostError('[22:19:43] C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/gbm/gbtree.cc:597: Check failed: common::AllVisibleGPUs() >= 1 (0 vs. 1) : No visible GPU is found for XGBoost.')
Traceback (most recent call last):
  File "C:\Users\RAKSHANDA\anaconda3\lib\site-packages\optuna\study\_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\RAKSHA~1\AppData\Local\Temp/ipykernel_12528/3398674160.py", line 17, in objective
    xgb_reg_model.fit(train_x,train_y,eval_set = [(test_x,test_y)] , verbose = True )
  File "C:\Users\RAKSHANDA\anaconda3\lib\site-packages\xgboost\core.py", line 532, in inner_f
    return f(**kwargs)
  File "C:\Users\RAKSHANDA\anaconda3\lib\site-packages\xgboost\sklearn.py", line 961, in fit
    self._Booster = train(
  File "C

[22:19:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:248: No visible GPU is found, setting `gpu_id` to -1
[22:19:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBoostError: [22:19:43] C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/gbm/gbtree.cc:597: Check failed: common::AllVisibleGPUs() >= 1 (0 vs. 1) : No visible GPU is found for XGBoost.